In [ ]:
import os
import subprocess
import threading

os.environ['HF_HOME']='/srv/starter_content/cache'

In [ ]:
model = "eci-io/climategpt-7b"

## NDP LLM Service Documentation

This Python code snippet is designed to launch various components of a chat service named "FastChat." Each function starts a different part of the service using the `subprocess.run` method to execute shell commands.

### `run_controller()`

Starts the controller for the FastChat service, responsible for managing and coordinating different parts of the service.

```python
def run_controller():
    subprocess.run(["python3", "-m", "fastchat.serve.controller", "--host", "127.0.0.1"])


In [ ]:
def run_controller():
    subprocess.run(["python3", "-m", "fastchat.serve.controller", "--host", "127.0.0.1"])

## `run_worker`
Initiates a model worker for processing and generating responses based on specified models. Runs the model worker module, specifying the local host and a list of model names for processing requests. The --model-path argument should point to the directory where the models are stored.
```python 
def run_model_worker():
    subprocess.run(["python3", "-m", "fastchat.serve.model_worker", "--host", "127.0.0.1", "--model-names", "gpt-4-turbo,Trendyol-LLM-7b-chat-v0.1,text-embedding-ada-002", "--model-path", model])

```
### `run_api`

Launches an API server that handles API requests to the FastChat service.
Runs the API server module on the local host, acting as an interface between the service and external clients or applications.
```python
def run_api_server():
    subprocess.run(["python3", "-m", "fastchat.serve.openai_api_server", "--host", "127.0.0.1"])
```    


In [ ]:
def run_model_worker():
    subprocess.run(["python3", "-m", "fastchat.serve.model_worker", "--host", "127.0.0.1", "--model-names", "eci-io/climategpt-7b,text-embedding-ada-002", "--model-path", model])

def run_api_server():
    subprocess.run(["python3", "-m", "fastchat.serve.openai_api_server", "--host", "127.0.0.1"])
def run_ui_server():
    subprocess.run(["python3", "-m", "fastchat.serve.gradio_web_server", "--host", "127.0.0.1"])


## Starting the `run_controller` Function in a Separate Thread

To enable the FastChat controller to run concurrently with the main program, the `run_controller` function is executed in a separate thread. This is achieved using Python's `threading` module, which allows for the execution of code in parallel to the main execution flow of the program.

### Code Snippet:

```python
import threading

controller_thread = threading.Thread(target=run_controller)
controller_thread.start()
```

In [ ]:
controller_thread = threading.Thread(target=run_controller)
controller_thread.start()

## Starting the `run_model_worker` Function in a Separate Thread

To facilitate concurrent execution of the FastChat model worker alongside the main program and potentially other service components, the `run_model_worker` function is executed in a separate thread. This concurrent execution is made possible through the use of Python's `threading` module.

### Code Snippet:

```python
import threading

model_worker_thread = threading.Thread(target=run_model_worker)
model_worker_thread.start()
```

In [ ]:
# !pip install langchain openai chromadb

In [ ]:
model_worker_thread = threading.Thread(target=run_model_worker)
model_worker_thread.start()

## Running the `run_api_server` Function in a Separate Thread

To ensure the API server component of the FastChat service operates concurrently with other parts of the application, the `run_api_server` function is launched in a separate thread. This concurrency is achieved with the help of Python's `threading` module, allowing multiple components to run simultaneously, improving scalability and responsiveness.

### Code Snippet:

```python
import threading

api_server_thread = threading.Thread(target=run_api_server)
api_server_thread.start()


In [ ]:
api_server_thread = threading.Thread(target=run_api_server)
api_server_thread.start()

In [ ]:
!curl  -X 'GET' \
  'http://localhost:8000/v1/models' \
  -H 'accept: application/json'

In [ ]:
!curl http://localhost:8000/v1/chat/completions  -H "Content-Type: application/json"  -d '{ "model": "eci-io/climategpt-7b", "messages": [{"role": "user", "content": "Who is Fumio Kishida?"}]}'

In [ ]:
import requests

url = "http://localhost:8000/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
}

data = {
    "model": "eci-io/climategpt-7b",
    "messages": [{"role": "user", "content": "Who is Fumio Kishida?"}],
}

response = requests.post(url, json=data, headers=headers)

print(response.text)


In [ ]:
import requests

url = "http://localhost:8000/api/v1/chat/completions"

headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
}

data = {
    "model": "eci-io/climategpt-7b",
    "messages": "What did the president say about Ketanji Brown Jackson?",
    "temperature": 0.7,
    "top_p": 1,
    "top_k": -1,
    "n": 1,
    "max_tokens": 650,
    "stop": "string",
    "stream": False,
    "user": "string",
    "repetition_penalty": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
}

response = requests.post(url, json=data, headers=headers)

print(response.text)


In [ ]:
os.environ['OPENAI_API_BASE'] = 'http://localhost:8000/v1'
os.environ['OPENAI_API_KEY'] = 'EMPTY'

In [ ]:
!wget https://raw.githubusercontent.com/hwchase17/langchain/v0.0.200/docs/modules/state_of_the_union.txt

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
loader = TextLoader("state_of_the_union.txt")
index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
llm = ChatOpenAI(model="eci-io/climategpt-7b")

questions = [
    "Who is the speaker",
    "What did the president say about Ketanji Brown Jackson",
    "What are the threats to America",
    "Who are mentioned in the speech",
    "Who is the vice president",
    "How many projects were announced",
]

for query in questions:
    print("Query:", query)
    print("Answer:", index.query(query, llm=llm))

In [ ]:
print("Answer:", index.query("Where is San Diego?", llm=llm))

In [ ]:
print("Answer:", index.query("I want to go to San Diego from St. George Utah give me a plan", llm=llm))

In [ ]:
print("Answer:", index.query("Who is narendra modi?", llm=llm))